In [58]:
import os
import json
from glob import glob
from tqdm import tqdm
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
from datetime import datetime, timedelta

from google.cloud import bigquery
from google.oauth2 import service_account

pd.set_option('display.max_seq_items', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

#pd.set_option('display.max_rows',None)     # 모든 행 보기
#pd.reset_option('display.max_rows')        # 기본값으로 되돌리기

In [59]:
product36 = pd.read_csv('product36.csv')

In [60]:
# 매번 필요
def pad_with_zeros(string):
    """
    문자열의 길이가 19이 될 때까지 앞에 0을 추가하는 함수
    """
    # 문자열의 길이를 세기
    length = len(string)
    
    # 문자열의 길이가 20이 될 때까지 앞에 0을 추가
    while length < 19:
        string = '0' + string
        length += 1
    
    return string


product36['fullVisitorId'] = product36['fullVisitorId'].astype(str)
product36['fullVisitorId'] = product36['fullVisitorId'].apply(pad_with_zeros)

# 아래는 사전에 완료한 전처리
# product = product.assign(hits_product_productPrice = product.hits_product_productPrice/1000000)
# product = product.assign(hits_product_productRevenue = product.hits_product_productRevenue/1000000)
# product['hits_product_productRevenue'] = product['hits_product_productRevenue'].round(2)
# product.loc[(product.hits_eCommerceAction_action_type==6) & (product.hits_transaction_transactionRevenue.isnull()), 'hits_eCommerceAction_action_type'] = 0
# product = product.drop(['Unnamed: 0', 'hits_transaction_transactionRevenue'], axis=1)

In [151]:
# 상품 이름을 보고 카테고리 분류
def name_categorize_category(row):
    if any(keyword in row['hits_product_v2ProductName'] for keyword in ['Toddler','Youth','Women','Men','Bib', 'Dress Socks','Vintage Henley','Tee', 'Sweatshirt', 'Jacket', 'Cap', 'Zip', 'Sleeve' , 'Hoodie', 'Hat', 'Onesie', 'Vest', 'Baby', 'Shirt']):
        return 'Apparel'
    elif any(keyword in row['hits_product_v2ProductName'] for keyword in ['Set of 3 Nested Travel Cases','Foldaway Shopper','Chevron Shopper','Collapsible Duffel','Backpack','Electronics Accessory Pouch','Bag', 'Rucksack', 'Softsided Travel Pouch Set', 'Backpack', 'Tote']):
        return 'Bags' 
    elif any(keyword in row['hits_product_v2ProductName'] for keyword in ['Nest® Learning Thermostat 3rd Gen-USA','PowerKit','Headset','Google Device Holder Sticky Pad','Charger','Grip Kit Cable Organizer','Device Stand','Micro Charger','Phone Holder','Compact Selfie Stick','Waze Mobile Phone Vent Mount','Speaker', 'Compact Charger','Bluetooth', 'Flashlight', 'Earbuds', 'Power Bank']):
        return 'Electronics' 
    elif any(keyword in row['hits_product_v2ProductName'] for keyword in ['Android 24 oz Contigo Addison','Koozie Can Kooler','Bottle', 'Mug', 'Tumbler', 'Cups']):
        return 'Drinkware'
    elif any(keyword in row['hits_product_v2ProductName'] for keyword in ['Set of 3 Packing Cubes','Google Badge Pull','Screen Cleaning Cloth','Notebook', 'Journal', 'Pen', 'Mouse Pad', 'Badge Holder', 'Galaxy Screen Cleaning Cloth']):
        return 'Office' 
    elif any(keyword in row['hits_product_v2ProductName'] for keyword in ['Gift Card']):
        return 'Gift Card' 
    elif any(keyword in row['hits_product_v2ProductName'] for keyword in ['Android Cool Gear Lunch To Go','Google 7-inch Dog Flying Disc','Pop-a-Point Crayon','Yoga Block','Android Lunch Kit','9 Decal Set','Beach Mat','Yoga Mat','Pet Feeding Mat','Luggage Tag','Dog Frisbee','Waze Mood Window Decal','Kick Ball','Seat Pack Organizer','Pet Bowl','Sanitizer','Windup Android','Umbrella','Sticker', 'Dog Toy', 'Doodle Decal', 'Custom Decals', '7" Dog Frisbee', 'Lip Balm', 'Suitcase Organizer Cubes', 'Sunglasses']):
        return 'Accessories' 
    else:
        return row['name_category']

product36['name_category'] = product36.apply(name_categorize_category, axis=1)

In [ ]:
# Apparel        194  27.56달러
# Accessories    39   3.82
# Office         65   5.36
# Drinkware      37   11.28
# Bags           29   35.63
# Electronics    35   29.91
# Gift Card      4 

In [8]:
# # new_category는 hits_product_v2ProductCategory를 참고로 1차 생성
# def categorize_category1(category):
#     category = category.split('/')
#     if 'Office' in category or 'Notebooks & Journals' in category:
#         return 'Office'
#     elif 'Apparel' in category or "Men's" in category or "Headgear" in category or "Men's-Outerwear" in category or "Mens Outerwear" in category or "Fruit Games" in category or "Kids" in category:
#         return 'Apparel'
#     elif 'Drinkware' in category or 'Mugs' in category or 'Bottles' in category or 'Tumblers' in category:
#         return 'Drinkware'
#     elif 'Accessories' in category or 'Fun' in category or 'Housewares' in category:
#         return 'Accessories'
#     elif 'Lifestyle' in category:
#         return 'Lifestyle'
#     elif 'Electronics' in category or 'Wearables' in category:
#         return 'Electronics'
#     elif 'Gift Cards' in category:
#         return 'Gift Card'
#     elif 'Shop by Brand' in category or 'Brands' in category or 'YouTube' in category or 'Google' in category or 'Android' in category or 'Waze' in category:
#         return 'Shop by Brand'
#     elif 'Bags' in category or'More Bags' in category or 'Backpacks' in category:
#         return 'Bags'
#     # elif 'Spring Sale!' in category or 'Limited Supply' in category or 'Clearance Sale' in category:
#     #     return 'Promotion'
#     else:
#         return 'Other'
# product36['new_category'] = product36['hits_product_v2ProductCategory'].apply(categorize_category1)

In [9]:
# # 두 가지 카테고리에 속하면 제외 / new_category의 other을 줄인 [new_category2 생성]
# def categorize_category2(row):
#     if row['new_category'] == 'Other' and any(keyword in row['hits_product_v2ProductName'] for keyword in ['Yoga Pants','Eco-Jersey','Back Tank Black','T-shirt','Youth Baseball Raglan','Baby Esssentials Set','Polo','Bib','Vest','Hat','Vintage','Tank Top','Quilted Insulated','Stretch Fit Hat','Baby Essentials','Google Canvas Tote Fruit Games Lemon','Cap', 'Tee', 'Sleeve', 'Sweatshirt', 'Vintage Henley', 'Hoodie', 'Dress Socks', 'Zip', 'Onesie(4)', 'Onesie', 'Snapback Hat', 'Shirt', 'Jacket', 'Vintage Tank']):
#         return 'Apparel'
#     elif row['new_category'] == 'Other' and any(keyword in row['hits_product_v2ProductName'] for keyword in ['Dog Frisbee','Umbrella']):
#         return 'Lifestyle'
#     elif row['new_category'] == 'Other' and any(keyword in row['hits_product_v2ProductName'] for keyword in ['Collapsible Duffel','Air Mesh Tote','Shopper','Sport Bag', 'Travel Bag','Shopping Bag','Tote Bag', 'Backpack', 'Google Canvas Tote Natural/Navy', 'Rucksack', 'Collapsible Duffel']):
#         return 'Bags' 
#     elif row['new_category'] == 'Other' and any(keyword in row['hits_product_v2ProductName'] for keyword in ['Earbud','Phone Vent Mount','Google Phone Sanitizer','Smart Speaker','Bluetooth','Recycled Mouse Pad','Holder Sticky Pad','Flashlight', 'Bluetooth Speaker', 'Power Bank', 'Clip Phone Holder', 'Earbuds', 'Bluetooth Headphones', 'PowerKit', 'Device Stand', 'Charger']):
#         return 'Electronics' 
#     elif row['new_category'] == 'Other' and any(keyword in row['hits_product_v2ProductName'] for keyword in ['Cups-Sets','Tumbler','Bottle', 'Mug', 'Wide Mouth Sport Bottle']):
#         return 'Drinkware'
#     elif row['new_category'] == 'Other' and any(keyword in row['hits_product_v2ProductName'] for keyword in ['Badge Pull','Pen','Grip Pen','Pencil Set','Gel Roller Pen','Pen Pencil & Highlighter Set','Maze Pen','Badge Holder', 'Metal Texture Roller Pen','Journal', 'Retractable Pen', 'Stylus Pen w/ LED Light', 'Notebook', 'Ballpoint Pen']):
#         return 'Office' 
#     elif row['new_category'] == 'Other' and any(keyword in row['hits_product_v2ProductName'] for keyword in ['Gift Card']):
#         return 'Gift Card' 
#     elif row['new_category'] == 'Other' and any(keyword in row['hits_product_v2ProductName'] for keyword in ['Luggage Tag','Bike Saddle Bag','Waze Pack of 9 Decal Set','Waze Baby on Board Window Decal','Waze Mood Window Decal(3)', 'Custom Decals']):
#         return 'Accessories' 
#     # elif row['new_category'] == 'Other' and any(keyword in row['hits_product_v2ProductName'] for keyword in []):
#     #     return 'Limited Supply' 
#     # elif row['new_category'] == 'Other' and any(keyword in row['hits_product_v2ProductName'] for keyword in ['7" Dog Frisbee','DOT Sticker','8pc Android Sticker Sheet','Gen','YouTube Sticker Sheet','Contigo Addison','Packing Cubes','YouTube Sticker Sheet','Kick Ball','Handlebar Bag','7-inch Dog Flying','Travel Pouch','Feeding Mat','Can Kooler','Android Lunch Kit','Selfie Stick','Pop-a-Point Crayon','Cleaning Cloth','Gear Bag','Pet Bowl','Yoga Mat','Android Sticker Sheet Ultra Removable','Cable Organizer','Ultra Removable','Organizer Cubes','Grip Kit Cable Organizer','Laptop and Cell Phone Stickers','Seat Pack Organizer','Dog Toy','Lip Balm','Doodle Decal','1 oz Hand Sanitizer','Google Kick Ball','Google Lunch Bag','Fashion Sunglasses & Pouch','Accessory Pouch','Windup','Sunglasses(3)', 'Straw Beach Mat']):
#     #     return 'delete' 
#     else:
#         return row['new_category']

# product36['final_category'] = product36.apply(categorize_category2, axis=1)

In [10]:
# # 두 가지 카테고리에 속하면 제외 / new_category2에서 shop by shop을 재분배
# def categorize_category3(row):
#     if row['final_category'] == 'Shop by Brand' and any(keyword in row['hits_product_v2ProductName'] for keyword in ['Polo','Baby Essentials','Shirt','Yoga Pants','Performance Pullover','Back Tank','Bib','Rucksack',"Android Men's Vintage",'Eco-Jersey','Onesie','Quilted Insulated','Jacket','Vintage Tank','Vintage Henley','Cap','Tee','Dress Socks','Sleeve','Hat','Hoodie']):
#         return 'Apparel'
#     elif row['final_category'] == 'Shop by Brand' and any(keyword in row['hits_product_v2ProductName'] for keyword in ['Umbrella']):
#         return 'Lifestyle'
#     elif row['final_category'] == 'Shop by Brand' and any(keyword in row['hits_product_v2ProductName'] for keyword in ['Style Backpack','Waterproof Backpack']):
#         return 'Bags' 
#     elif row['final_category'] == 'Shop by Brand' and any(keyword in row['hits_product_v2ProductName'] for keyword in ['PowerKit','Power Bank','Bluetooth','Device Stand','Mobile Phone Vent Mount']):
#         return 'Electronics' 
#     elif row['final_category'] == 'Shop by Brand' and any(keyword in row['hits_product_v2ProductName'] for keyword in ['Mug','Bottle', 'Tumbler']):
#         return 'Drinkware'
#     elif row['final_category'] == 'Shop by Brand' and any(keyword in row['hits_product_v2ProductName'] for keyword in ['Pen w/ LED Light','Hard Cover Journal','Journal with Pen','Notebook','Journal Book Set','RFID Journa']):
#         return 'Office' 
#     elif row['final_category'] == 'Shop by Brand' and any(keyword in row['hits_product_v2ProductName'] for keyword in ['Waze Pack of 9 Decal Set','Window Decal','Luggage Tag']):
#         return 'Accessories' 
#     elif row['final_category'] == 'Shop by Brand' and any(keyword in row['hits_product_v2ProductName'] for keyword in ['Doodle Decal','Collapsible Pet Bowl','Google Car Clip Phone Holder','Lunch Kit','Google Laptop and Cell Phone Stickers','Sticker Sheet','Windup','Phone Stickers','Custom Decals']):
#         return 'Other' 
#     else:
#         return row['final_category']
    

# product36['final_category'] = product36.apply(categorize_category3, axis=1)

In [11]:
'''
안드로이드 55개
구글 176개
waze 9개
유튜브 38개

Bags  31개
Accessories 49개
Drinkware 37개
Apparel 197개
office 77개
Electronics 47개
Lifestyle 39개

'''

'\n안드로이드 55개\n구글 176개\nwaze 9개\n유튜브 38개\n\nBags  31개\nAccessories 49개\nDrinkware 37개\nApparel 197개\noffice 77개\nElectronics 47개\nLifestyle 39개\n\n'

In [12]:
# # (shop by shop 필요없음)  두 가지 카테고리에 속하면 제외 / new_category2에서 shop by shop을 재분배
# def categorize_brand(product_name):
#     category = product_name
#     category_list = category.split()
#     if 'Google' in category_list:
#         return 'Google'
#     elif 'YouTube' in category_list:
#         return 'YouTube'
#     elif 'Android' in category_list:
#         return 'Android' 
#     elif 'Waze' in category_list:
#         return 'Waze'
#     else:
#         return '(not brand)'
    

# product36['brand_category'] = product36['hits_product_v2ProductName'].apply(categorize_brand)

In [14]:
# product['hits_product_v2ProductName'] = product['hits_product_v2ProductName'].replace("YouTube Women's Short Sleeve Tri-blend Badge Tee.*", "YouTube Women's Short Sleeve Tri-blend Badge Tee(2)", regex=True)
# product['hits_product_v2ProductName'] = product['hits_product_v2ProductName'].replace("YouTube Men's Short Sleeve Hero Tee.*", "YouTube Men's Short Sleeve Hero Tee(3)", regex=True)
# product['hits_product_v2ProductName'] = product['hits_product_v2ProductName'].replace("Yoga Mat.*", "Yoga Mat(2)", regex=True)
# product['hits_product_v2ProductName'] = product['hits_product_v2ProductName'].replace("Waze Mood.*", "Waze Mood Window Decal(3)", regex=True)
# product['hits_product_v2ProductName'] = product['hits_product_v2ProductName'].replace("Waterpoof Gear Bag", "Waterproof Gear Bag", regex=True)
# product['hits_product_v2ProductName'] = product['hits_product_v2ProductName'].replace("PaperMate Ink Joy.*", "PaperMate Ink Joy Pen(2)", regex=True)
# product['hits_product_v2ProductName'] = product['hits_product_v2ProductName'].replace("Micro Wireless Earbud", "Micro Wireless Earbuds", regex=True)
# product['hits_product_v2ProductName'] = product['hits_product_v2ProductName'].replace("Google Youth Tee Fruit Games.*", "Google Youth Tee Fruit Games(4)", regex=True)
# product['hits_product_v2ProductName'] = product['hits_product_v2ProductName'].replace("Google Youth Short Sleeve Tee.*", "Google Youth Short Sleeve Tee(2)", regex=True)
# product['hits_product_v2ProductName'] = product['hits_product_v2ProductName'].replace("Google Youth Short Sleeve T-shirt.*", "Google Youth Short Sleeve T-shirt(2)", regex=True)
# product['hits_product_v2ProductName'] = product['hits_product_v2ProductName'].replace("Google Women's Vintage Hero Tee.*", "Google Women's Vintage Hero Tee(4)", regex=True)
# product['hits_product_v2ProductName'] = product['hits_product_v2ProductName'].replace("Google Women's Tee.*", "Google Women's Tee(2)", regex=True)
# product['hits_product_v2ProductName'] = product['hits_product_v2ProductName'].replace("Google Women's Short Sleeve V-Neck Tee.*", "Google Women's Short Sleeve V-Neck Tee(4)", regex=True)
# product['hits_product_v2ProductName'] = product['hits_product_v2ProductName'].replace("Google Women's Short Sleeve Shirt.*", "Google Women's Short Sleeve Shirt(3)", regex=True)
# product['hits_product_v2ProductName'] = product['hits_product_v2ProductName'].replace("Google Women's Short Sleeve Performance Tee.*", "Google Women's Short Sleeve Performance Tee(4)", regex=True)
# product['hits_product_v2ProductName'] = product['hits_product_v2ProductName'].replace("Google Women's Short Sleeve Hero Tee.*", "Google Women's Short Sleeve Hero Tee(6)", regex=True)
# product['hits_product_v2ProductName'] = product['hits_product_v2ProductName'].replace("Google Women's Short Sleeve Badge Tee.*", "Google Women's Short Sleeve Badge Tee(3)", regex=True)
# product['hits_product_v2ProductName'] = product['hits_product_v2ProductName'].replace("Google Women's Scoop Neck Tee.*", "Google Women's Scoop Neck Tee(3)", regex=True)
# product['hits_product_v2ProductName'] = product['hits_product_v2ProductName'].replace("Google Women's Quilted Insulated.*", "Google Women's Quilted Insulated(3)", regex=True)
# product['hits_product_v2ProductName'] = product['hits_product_v2ProductName'].replace("Google Women's 1/4 Zip Performance Pullover.*", "Google Women's 1/4 Zip Performance Pullover(2)", regex=True)
# product['hits_product_v2ProductName'] = product['hits_product_v2ProductName'].replace("Google Toddler Tee Fruit Games.*", "Google Toddler Tee Fruit Games(4)", regex=True)
# product['hits_product_v2ProductName'] = product['hits_product_v2ProductName'].replace("Google Toddler Short Sleeve Tee.*", "Google Toddler Short Sleeve Tee(2)", regex=True)
# product['hits_product_v2ProductName'] = product['hits_product_v2ProductName'].replace("Google Toddler Short Sleeve T-shirt.*", "Google Toddler Short Sleeve T-shirt(4)", regex=True)
# product['hits_product_v2ProductName'] = product['hits_product_v2ProductName'].replace("Google Sunglasses.*", "Google Sunglasses(3)", regex=True)
# product['hits_product_v2ProductName'] = product['hits_product_v2ProductName'].replace("Google Stretch Fit Hat.*", "Google Stretch Fit Hat Navy", regex=True)
# product['hits_product_v2ProductName'] = product['hits_product_v2ProductName'].replace("Google Onesie.*", "Google Onesie(4)", regex=True)
# product['hits_product_v2ProductName'] = product['hits_product_v2ProductName'].replace("Google Men's Vintage Badge Tee.*", "Google Men's Vintage Badge Tee(4)", regex=True)
# product['hits_product_v2ProductName'] = product['hits_product_v2ProductName'].replace("Google Men's Skater Tee.*", "Google Men's Skater Tee(2)", regex=True)
# product['hits_product_v2ProductName'] = product['hits_product_v2ProductName'].replace("Google Men's Short Sleeve Performance Badge.*", "Google Men's Short Sleeve Performance Badge(4)", regex=True)
# product['hits_product_v2ProductName'] = product['hits_product_v2ProductName'].replace("Google Men's Short Sleeve Hero Tee.*", "Google Men's Short Sleeve Hero Tee(3)", regex=True)
# product['hits_product_v2ProductName'] = product['hits_product_v2ProductName'].replace("Google Men's Pullover Hoodie.*", "Google Men's Pullover Hoodie(2)", regex=True)
# product['hits_product_v2ProductName'] = product['hits_product_v2ProductName'].replace("Google Men's Performance Badge Tee.*", "Google Men's Performance Badge Tee(2)", regex=True)
# product['hits_product_v2ProductName'] = product['hits_product_v2ProductName'].replace("Google Men's Performance 1/4 Zip Pullover.*", "Google Men's Performance 1/4 Zip Pullover(3)", regex=True)
# product['hits_product_v2ProductName'] = product['hits_product_v2ProductName'].replace("Google Men's Long Sleeve Raglan.*", "Google Men's Long Sleeve Raglan(2)", regex=True)
# product['hits_product_v2ProductName'] = product['hits_product_v2ProductName'].replace("Google Men's Long & Lean Tee.*", "Google Men's Long & Lean Tee(2)", regex=True)
# product['hits_product_v2ProductName'] = product['hits_product_v2ProductName'].replace("Google Men's Heavyweight Long Sleeve Hero Tee.*", "Google Men's Heavyweight Long Sleeve Hero Tee(2)", regex=True)
# product['hits_product_v2ProductName'] = product['hits_product_v2ProductName'].replace("Google Men's Airflow 1/4 Zip Pullover.*", "Google Men's Airflow 1/4 Zip Pullover(2)", regex=True)
# product['hits_product_v2ProductName'] = product['hits_product_v2ProductName'].replace("Google Men's 100% Cotton Short Sleeve Hero Tee.*", "Google Men's 100% Cotton Short Sleeve Hero Tee(4)", regex=True)
# product['hits_product_v2ProductName'] = product['hits_product_v2ProductName'].replace("Google Leather Journal.*", "Google Leather Journal(3)", regex=True)
# product['hits_product_v2ProductName'] = product['hits_product_v2ProductName'].replace("Google Infant Zip Hood.*", "Google Infant Zip Hood(3)", regex=True)
# product['hits_product_v2ProductName'] = product['hits_product_v2ProductName'].replace("Google Infant Short Sleeve Tee.*", "Google Infant Short Sleeve Tee(4)", regex=True)
# product['hits_product_v2ProductName'] = product['hits_product_v2ProductName'].replace("Google Heavyweight Long Sleeve Hero Tee.*", "Google Heavyweight Long Sleeve Hero Tee(2)", regex=True)
# product['hits_product_v2ProductName'] = product['hits_product_v2ProductName'].replace("Google Collapsible Pet Bowl.*", "Google Collapsible Pet Bowl(2)", regex=True)
# product['hits_product_v2ProductName'] = product['hits_product_v2ProductName'].replace("Google Collapsible Duffel.*", "Google Collapsible Duffel(2)", regex=True)
# product['hits_product_v2ProductName'] = product['hits_product_v2ProductName'].replace("Google Bib.*", "Google Bib White(2)", regex=True)
# product['hits_product_v2ProductName'] = product['hits_product_v2ProductName'].replace("Google Adult Tee Fruit Games.*", "Google Adult Tee Fruit Games(3)", regex=True)
# product['hits_product_v2ProductName'] = product['hits_product_v2ProductName'].replace("Google 25 oz.*", "Google 25 oz Stainless Steel Bottle(3)", regex=True)
# product['hits_product_v2ProductName'] = product['hits_product_v2ProductName'].replace("Eco-Aware Large Recycled Journal Combo", "Eco-Aware Recycled Journal Combo", regex=True)
# product['hits_product_v2ProductName'] = product['hits_product_v2ProductName'].replace("Android Youth Short Sleeve T-shirt.*", "Android Youth Short Sleeve T-shirt(2)", regex=True)
# product['hits_product_v2ProductName'] = product['hits_product_v2ProductName'].replace("Android Women's Short Sleeve Tri-blend Badge Tee.*", "Android Women's Short Sleeve Tri-blend Badge Tee(2)", regex=True)
# product['hits_product_v2ProductName'] = product['hits_product_v2ProductName'].replace("Android Women's Short Sleeve Badge Tee Light.*", "Android Women's Short Sleeve Badge Tee Light(2)", regex=True)
# product['hits_product_v2ProductName'] = product['hits_product_v2ProductName'].replace("Android Women's Short Sleeve Badge.*", "Android Women's Short Sleeve Badge Tee(2)", regex=True)
# product['hits_product_v2ProductName'] = product['hits_product_v2ProductName'].replace("Android Toddler Short Sleeve T-shirt.*", "Android Toddler Short Sleeve T-shirt(3)", regex=True)
# product['hits_product_v2ProductName'] = product['hits_product_v2ProductName'].replace("Android Stretch Fit Hat.*", "Android Stretch Fit Hat", regex=True)
# product['hits_product_v2ProductName'] = product['hits_product_v2ProductName'].replace("Android Onesie.*", "Android Onesie(3)", regex=True)
# product['hits_product_v2ProductName'] = product['hits_product_v2ProductName'].replace("Android Men's Vintage.*", "Android Men's Vintage(3)", regex=True)
# product['hits_product_v2ProductName'] = product['hits_product_v2ProductName'].replace("Android Men's Short Sleeve Hero Tee.*", "Android Men's Short Sleeve Hero Tee(2)", regex=True)
# product['hits_product_v2ProductName'] = product['hits_product_v2ProductName'].replace("Android Infant Short Sleeve Tee.*", "Android Infant Short Sleeve Tee(2)", regex=True)
# product['hits_product_v2ProductName'] = product['hits_product_v2ProductName'].replace("8 pc Android Sticker Sheet", "8pc Android Sticker Sheet", regex=True)
# product['hits_product_v2ProductName'] = product['hits_product_v2ProductName'].replace("7&quot; Dog Frisbee", "Dog Frisbee", regex=True)
# product['hits_product_v2ProductName'] = product['hits_product_v2ProductName'].replace("16 oz. Hot and Cold Tumbler", "16 oz. Hot/Cold Tumbler", regex=True)
